In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('enron_emails.csv')
df.head()

,Message-ID,From,To,Date,content,clean_content
0,<8345058.1075840404046.JavaMail.evans@thyme>,('advdfeedback@investools.com'),('advdfeedback@investools.com'),2002-01-29 23:20:55,INVESTools Advisory\r\nA Free Digest of Truste...,investools advisory free digest trusted invest...
1,<1512159.1075863666797.JavaMail.evans@thyme>,('richard.sanders@enron.com'),('richard.sanders@enron.com'),2000-09-20 19:07:00,----- Forwarded by Richard B Sanders/HOU/ECT o...,forwarded richard b sanders hou ect pm justin ...
2,<26118676.1075862176383.JavaMail.evans@thyme>,('m..love@enron.com'),('m..love@enron.com'),2001-10-30 16:15:17,hey you are not wearing your target purple shi...,hey wearing target purple shirt today mine wan...
3,<10369289.1075860831062.JavaMail.evans@thyme>,('leslie.milosevich@kp.org'),('leslie.milosevich@kp.org'),2002-01-30 17:54:18,Leslie Milosevich\r\n1042 Santa Clara Avenue\r...,leslie milosevich santa clara avenue alameda c...
4,<26728895.1075860815046.JavaMail.evans@thyme>,('rtwait@graphicaljazz.com'),('rtwait@graphicaljazz.com'),2002-01-30 19:36:01,"Rini Twait\r\n1010 E 5th Ave\r\nLongmont, CO 8...",rini twait e th ave longmont co rtwait graphic...


In [9]:
df.drop('clean_content', inplace=True, axis=1)

## Word tokenisation

In [10]:
from nltk import word_tokenize
import re

In [11]:
# # Remove whitespaces
# text = df['content'].str.lstrip()
# text = text.str.rstrip()

# # Replace with lowercase
# text = text.str.lower()

# # Tokenize
# text = text.apply(lambda x: word_tokenize(x))

In [16]:
# Tokenize
text = df.apply(lambda x: word_tokenize(x['content']), axis=1)

# Remove whitespaces
text = text.apply(lambda x: [w.lstrip() for w in x])
text = text.apply(lambda x: [w.rstrip() for w in x])

# Switch all to lowercase
text = text.apply(lambda x: [w.lower() for w in x])
text

0       [investools, advisory, a, free, digest, of, tr...
1       [--, --, -, forwarded, by, richard, b, sanders...
2       [hey, you, are, not, wearing, your, target, pu...
3       [leslie, milosevich, 1042, santa, clara, avenu...
4       [rini, twait, 1010, e, 5th, ave, longmont, ,, ...
                              ...                        
2085    [i, bot, 1,000/d, at, 3.175, apr/oct02, ., put...
2086                   [i, 'm, okay, ., how, are, you, ?]
2087    [[, image, ], =09, [, image, ], [, image, ], [...
2088    [transmission, expansion, and, systems, in, tr...
2089    [sorry, nella, ,, i, replied, a, while, ago, d...
Length: 2090, dtype: object

## Remove all digits, punctuation, and stopwords from strings

In [23]:
from nltk.corpus import stopwords
import string

In [60]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [61]:
def remove_single_char(string, remove):
    myStr = ""
    for s in string:
        if s not in remove:
            myStr = myStr + s
            
    return myStr

def remove_digit(string):
    myStr = ""
    for s in string:
        if not s.isdigit():
            myStr = myStr + s
            
    return myStr

print(remove_single_char('work..!@#$%^&*()}{":?><not', exclude))

print(remove_digit('apr/oct02'))

worknot
apr/oct


In [62]:
# remove digits
digit_free = text.apply(lambda x: [remove_digit(w) for w in x])
digit_free

0       [investools, advisory, a, free, digest, of, tr...
1       [--, --, -, forwarded, by, richard, b, sanders...
2       [hey, you, are, not, wearing, your, target, pu...
3       [leslie, milosevich, , santa, clara, avenue, a...
4       [rini, twait, , e, th, ave, longmont, ,, co, ,...
                              ...                        
2085    [i, bot, ,/d, at, ., apr/oct, ., put, it, agai...
2086                   [i, 'm, okay, ., how, are, you, ?]
2087    [[, image, ], =, [, image, ], [, image, ], [, ...
2088    [transmission, expansion, and, systems, in, tr...
2089    [sorry, nella, ,, i, replied, a, while, ago, d...
Length: 2090, dtype: object

In [63]:
# remove punctuation
punc_free = digit_free.apply(lambda x: [remove_single_char(w, exclude) for w in x
                                       ])
punc_free

0       [investools, advisory, a, free, digest, of, tr...
1       [, , , forwarded, by, richard, b, sandershouec...
2       [hey, you, are, not, wearing, your, target, pu...
3       [leslie, milosevich, , santa, clara, avenue, a...
4       [rini, twait, , e, th, ave, longmont, , co, , ...
                              ...                        
2085    [i, bot, d, at, , aproct, , put, it, against, ...
2086                      [i, m, okay, , how, are, you, ]
2087    [, image, , , , image, , , image, , , image, ,...
2088    [transmission, expansion, and, systems, in, tr...
2089    [sorry, nella, , i, replied, a, while, ago, di...
Length: 2090, dtype: object

In [64]:
# remove stopwords
stop_free = punc_free.apply(lambda x: [w for w in x
                                       if w not in stop])
# remove empty strings
stop_free = stop_free.apply(lambda x: [w for w in 
                                       x if w!=''])
stop_free

0       [investools, advisory, free, digest, trusted, ...
1       [forwarded, richard, b, sandershouect, pm, jus...
2       [hey, wearing, target, purple, shirt, today, m...
3       [leslie, milosevich, santa, clara, avenue, ala...
4       [rini, twait, e, th, ave, longmont, co, rtwait...
                              ...                        
2085            [bot, aproct, put, digital, gasx, thanks]
2086                                               [okay]
2087    [image, image, image, image, image, image, ima...
2088    [transmission, expansion, systems, transition,...
2089    [sorry, nella, replied, ago, directly, rob, mi...
Length: 2090, dtype: object

In [65]:
sum(text.apply(lambda x: True if 'a' in x
          else False))

1424

In [66]:
sum(stop_free.apply(lambda x: True if 'a' in x
          else False))

0

### Removing special words and small words

Remove the word "image" since it comes from html tags

Remove words less than 3 chars in length

In [67]:
l1 = lambda x: True if x!='image' else False 

stop_free = stop_free.apply(lambda x: 
                              list(filter(l1, x)))

l1 = lambda x: True if len(x)>2 else False 

stop_free = stop_free.apply(lambda x: 
                              list(filter(l1, x)))

stop_free

0       [investools, advisory, free, digest, trusted, ...
1       [forwarded, richard, sandershouect, justin, bo...
2       [hey, wearing, target, purple, shirt, today, m...
3       [leslie, milosevich, santa, clara, avenue, ala...
4       [rini, twait, ave, longmont, rtwait, graphical...
                              ...                        
2085            [bot, aproct, put, digital, gasx, thanks]
2086                                               [okay]
2087    [syncrasy, llc, fax, texas, avenue, suite, ous...
2088    [transmission, expansion, systems, transition,...
2089    [sorry, nella, replied, ago, directly, rob, mi...
Length: 2090, dtype: object

## Lemmatize all words
Words in third person are changed to first person and verbs in past and future tenses are changed into present tense

In [70]:
from nltk.stem import WordNetLemmatizer

In [71]:
lemma = WordNetLemmatizer()


In [72]:
print(lemma.lemmatize('boxes', pos='v'))
print(lemma.lemmatize('went'))
print(lemma.lemmatize('went', pos='v'))

box
went
go


In [73]:
normalized = stop_free.apply(lambda x: [lemma.lemmatize(w, pos='v') for w in x])

In [74]:
normalized

0       [investools, advisory, free, digest, trust, in...
1       [forward, richard, sandershouect, justin, boyd...
2       [hey, wear, target, purple, shirt, today, mine...
3       [leslie, milosevich, santa, clara, avenue, ala...
4       [rini, twait, ave, longmont, rtwait, graphical...
                              ...                        
2085             [bot, aproct, put, digital, gasx, thank]
2086                                               [okay]
2087    [syncrasy, llc, fax, texas, avenue, suite, ous...
2088    [transmission, expansion, systems, transition,...
2089    [sorry, nella, reply, ago, directly, rob, miln...
Length: 2090, dtype: object

## Stemming
Words are reduced to their root form

In [75]:
from nltk.stem.snowball import SnowballStemmer
snow = SnowballStemmer(language='english')

In [76]:
cleaned_content = normalized.apply(lambda x: [snow.stem(w) for w in x])
cleaned_content

0       [investool, advisori, free, digest, trust, inv...
1       [forward, richard, sandershouect, justin, boyd...
2       [hey, wear, target, purpl, shirt, today, mine,...
3       [lesli, milosevich, santa, clara, avenu, alame...
4       [rini, twait, ave, longmont, rtwait, graphical...
                              ...                        
2085               [bot, aproct, put, digit, gasx, thank]
2086                                               [okay]
2087    [syncrasi, llc, fax, texa, avenu, suit, ouston...
2088    [transmiss, expans, system, transit, confer, f...
2089    [sorri, nella, repli, ago, direct, rob, milnth...
Length: 2090, dtype: object

In [77]:
# Convert the arrays back into single strings of text
df['cleaned_content'] = cleaned_content.apply(lambda x: " ".join(x))
df['cleaned_content'] = pd.Series(df['cleaned_content'].values, dtype=pd.StringDtype())

In [78]:
df.isna().sum()

Message-ID         0
From               0
To                 0
Date               0
content            0
cleaned_content    0
dtype: int64

In [79]:
df.to_csv('enron_emails_clean.csv', index=False)

In [80]:
df.shape

(2090, 6)